# Import 

In [72]:
import numpy as np
import pandas as pd
import torch 
from tqdm import tqdm
import matplotlib.pyplot as plt
from pathlib import Path
import os
from joblib import Parallel, delayed
from typing import Union, List


In [2]:
df_phenom = pd.read_parquet('/projects/synsight/data/openphenom/raw_well_embeddings/wells_em_openphenom.parquet')
df_meta = pd.read_csv('/projects/cpjump1/jump/metadata/complete_metadata.csv')

/tmp/ipykernel_4175210/599528953.py:2: DtypeWarning: Columns (2,8,9,10,11,12,13,14,15,18,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df_meta = pd.read_csv('/projects/cpjump1/jump/metadata/complete_metadata.csv')


In [73]:

df_dinov2_s = pd.read_parquet('/projects/synsight/data/dinov2_small/raw_well_embeddings/wells_em_dinov2s.parquet')
df_dinov2_g = pd.read_parquet('/projects/synsight/data/dinov2/raw_well_embeddings/wells_em_dinov2.parquet')
df_resnet = pd.read_parquet('/projects/cpjump1/jump/images_embeddings/resnet50_raw/metadata/load_wells/df_meta_raw_well.parquet')
df_chada = pd.read_parquet('/projects/synsight/data/chada/well_embeddings/metadata_raw_mean.parquet')


In [4]:
no_dmso_plates = [
            "Dest210823-174240",
            "Dest210628-162003",
            "Dest210823-174422",
        ]

In [5]:
base_path = Path('/projects/synsight/data/jump_embeddings/wells_embeddings')

# check cols

In [5]:
# List of all DataFrames
dataframes = {
    "df_phenom": df_phenom,
    "df_dinov2_s": df_dinov2_s,
    "df_dinov2_g": df_dinov2_g,
    "df_resnet": df_resnet,
    "df_chada": df_chada,
}

all_columns = set()
for name, df in dataframes.items():
    all_columns.update(df.columns)

column_presence = {col: {name: col in df.columns for name, df in dataframes.items()} for col in all_columns}
presence_df = pd.DataFrame(column_presence).T


NameError: name 'df_dinov2_s' is not defined

In [6]:
presence_df

NameError: name 'presence_df' is not defined

In [68]:
common_columns = set.intersection(*(set(df.columns) for df in dataframes.values()))
common_columns_df = pd.DataFrame({"Common Columns": list(common_columns)})
common_columns_df

,Common Columns
0,Metadata_Row
1,Metadata_Source
2,Metadata_Col
3,Metadata_Batch
4,Metadata_InChI
5,Metadata_Plate
6,Metadata_JCP2022
7,Metadata_Well


# Reference

In [6]:
images_df_path = "/projects/cpjump1/jump/load_data/final"
images_df = pd.read_parquet(images_df_path)

images_df["Metadata_Batch"] = images_df["Metadata_Batch"].astype(str)
df_meta["Metadata_Batch"] = df_meta["Metadata_Batch"].astype(str)
images_df["Metadata_Plate"] = images_df["Metadata_Plate"].astype(str)
df_meta["Metadata_Plate"] = df_meta["Metadata_Plate"].astype(str)
images_df["Metadata_Source"] = images_df["Metadata_Source"].astype(str)
df_meta["Metadata_Source"] = df_meta["Metadata_Source"].astype(str)
images_df["Metadata_Well"] = images_df["Metadata_Well"].astype(str)
df_meta["Metadata_Well"] = df_meta["Metadata_Well"].astype(str)

merged_df = pd.merge(images_df, df_meta, on=["Metadata_Source", "Metadata_Plate", "Metadata_Well"])
df = merged_df[merged_df["Metadata_PlateType"]=='COMPOUND'][['Metadata_Source', 'Metadata_Batch_x', 'Metadata_Plate',
       'Metadata_Well', 'Metadata_JCP2022', 'Metadata_InChI']]
df.columns = ['Metadata_Source', 'Metadata_Batch', 'Metadata_Plate', 'Metadata_Well', 'Metadata_JCP2022', 'Metadata_InChI']

In [28]:
ref_compound_df = df[~df['Metadata_JCP2022'].isin(['JCP2022_999999', 'JCP2022_UNKNOWN'])].drop_duplicates()[~df['Metadata_JCP2022'].isin(['JCP2022_999999', 'JCP2022_UNKNOWN'])].reset_index().drop(columns='index')

/tmp/ipykernel_4175210/1714066172.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ref_compound_df = df[~df['Metadata_JCP2022'].isin(['JCP2022_999999', 'JCP2022_UNKNOWN'])].drop_duplicates()[~df['Metadata_JCP2022'].isin(['JCP2022_999999', 'JCP2022_UNKNOWN'])].reset_index().drop(columns='index')


In [55]:
ref_compound_df = ref_compound_df[~ref_compound_df['Metadata_Plate'].isin(no_dmso_plates)].reset_index().drop(columns='index')

In [56]:
ref_compound_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 745884 entries, 0 to 745883
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Metadata_Source   745884 non-null  object
 1   Metadata_Batch    745884 non-null  object
 2   Metadata_Plate    745884 non-null  object
 3   Metadata_Well     745884 non-null  object
 4   Metadata_JCP2022  745884 non-null  object
 5   Metadata_InChI    745884 non-null  object
dtypes: object(6)
memory usage: 34.1+ MB


# pre-process

### function

In [10]:

def filter_and_save_embeddings_with_metadata(
    df: pd.DataFrame,
    ref_compound_df: pd.DataFrame,
    embedding_col: str,
    name: str,
    metadata_cols: List[str],
    storage_folder: Union[str, Path],
    num_jobs: int = -1,
) -> None:
    """
    Filters rows of a DataFrame to match a reference DataFrame, keeps relevant columns,
    loads embeddings into a single NumPy array, and saves metadata with matching indices.

    Args:
        df (pd.DataFrame): The DataFrame containing embedding paths and metadata.
        ref_compound_df (pd.DataFrame): Reference DataFrame to filter rows.
        embedding_col (str): The column containing paths to torch tensors.
        name (str): A name to identify the saved files.
        metadata_cols (List[str]): List of metadata columns to include.
        storage_folder (Union[str, Path]): Folder to save the outputs.
        num_jobs (int): Number of parallel jobs for embedding processing.
    """
    # Ensure the storage folder exists
    storage_folder = Path(storage_folder)
    storage_folder.mkdir(parents=True, exist_ok=True)

    # Validate embedding and metadata columns
    if embedding_col not in df.columns:
        raise ValueError(f"Column '{embedding_col}' not found in the DataFrame.")

    missing_metadata_cols = [col for col in metadata_cols if col not in df.columns]
    if missing_metadata_cols:
        raise ValueError(f"Missing metadata columns: {missing_metadata_cols}")

    # Filter rows based on the reference DataFrame
    filtered_df = df.merge(ref_compound_df, on=metadata_cols, how="inner")

    if filtered_df.empty:
        print("No matching rows found between the DataFrame and the reference DataFrame.")
        return

    # Function to load an embedding
    def load_embedding(embedding_path):
        try:
            tensor = torch.load(embedding_path, weights_only=True)
            return tensor.numpy()
        except Exception as e:
            print(f"Error loading embedding at {embedding_path}: {e}")
            return None

    # Wrap embedding loading with tqdm for progress tracking
    embedding_paths = filtered_df[embedding_col].tolist()

    # Use tqdm to wrap Parallel processing
    embeddings = Parallel(n_jobs=num_jobs)(
        delayed(lambda path: (path, load_embedding(path)))(path)
        for path in tqdm(embedding_paths)
    )
    embeddings = [emb for _, emb in embeddings if emb is not None]

    # Filter out failed embeddings and update metadata accordingly
    valid_indices = [i for i, emb in enumerate(embeddings) if emb is not None]
    embeddings = np.array([embeddings[i] for i in valid_indices])
    filtered_metadata = filtered_df.iloc[valid_indices].drop(columns=[embedding_col])[metadata_cols]

    # Save embeddings as a single NumPy array
    embeddings_path = storage_folder / f"embeddings_{name}.npy"
    np.save(embeddings_path, embeddings)

    # Save filtered metadata as a Parquet file
    metadata_parquet_path = storage_folder / f"metadata_{name}.parquet"
    filtered_metadata.to_parquet(metadata_parquet_path, index=False)

    print(f"Embeddings saved to: {embeddings_path}")
    print(f"Metadata saved to: {metadata_parquet_path}")
    return filtered_metadata, embeddings


# Example Usage
common_columns = [
    "Metadata_Source", "Metadata_Batch", "Metadata_Plate",
    "Metadata_Well", "Metadata_JCP2022", "Metadata_InChI"
]




### Resnet

In [11]:
resnet_path = base_path / 'resnet50'
df_resnet.columns

Index(['Metadata_Source', 'Metadata_Plate', 'Metadata_Well', 'Metadata_InChI',
       'Metadata_PlateType', 'Metadata_Batch', 'Well_Embedding_mean',
       'Well_Embedding_median', 'Metadata_Row', 'Metadata_Col',
       'Metadata_JCP2022'],
      dtype='object')

In [12]:
filtered_metadata, embeddings = filter_and_save_embeddings_with_metadata(
    df=df_resnet,
    ref_compound_df=ref_compound_df,
    embedding_col="Well_Embedding_mean",  
    name='resnet50_mean',
    metadata_cols=common_columns,
    storage_folder=resnet_path,
    num_jobs=79,
)

filtered_metadata.info()

100%|██████████| 745884/745884 [25:06<00:00, 495.02it/s] 


Embeddings saved to: /projects/synsight/data/jump_embeddings/wells_embeddings/resnet50/embeddings_resnet50_mean.npy
Metadata saved to: /projects/synsight/data/jump_embeddings/wells_embeddings/resnet50/metadata_resnet50_mean.parquet
<class 'pandas.core.frame.DataFrame'>
Index: 745884 entries, 0 to 745883
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Metadata_Source   745884 non-null  object
 1   Metadata_Batch    745884 non-null  object
 2   Metadata_Plate    745884 non-null  object
 3   Metadata_Well     745884 non-null  object
 4   Metadata_JCP2022  745884 non-null  object
 5   Metadata_InChI    745884 non-null  object
dtypes: object(6)
memory usage: 39.8+ MB


In [ ]:
filtered_metadata, embeddings = filter_and_save_embeddings_with_metadata(
    df=df_resnet,
    ref_compound_df=ref_compound_df,
    embedding_col="Well_Embedding_median",  
    name='resnet50_median',
    metadata_cols=common_columns,
    storage_folder=resnet_path,
    num_jobs=79,
)

filtered_metadata.info()

### chada

In [74]:
chada_path = base_path / 'chada'
df_chada.columns


Index(['Metadata_Source', 'Metadata_Plate', 'Metadata_Well', 'Metadata_InChI',
       'Metadata_PlateType', 'Metadata_Batch', 'Metadata_Is_Control',
       'Embeddings_mean', 'Metadata_Row', 'Metadata_Col', 'Path_Embeddings',
       'Metadata_JCP2022'],
      dtype='object')

In [76]:
df_chada = df_chada[~df_chada['Metadata_Plate'].isin(no_dmso_plates)].reset_index().drop(columns='index')

In [77]:
len(df_chada)

745879

In [78]:
# Drop the existing Metadata_JCP2022 column from df_chada if it exists
if 'Metadata_JCP2022' in df_chada.columns:
    df_chada = df_chada.drop(columns=['Metadata_JCP2022'])

# Merge df_chada with ref_compound_df to add the new Metadata_JCP2022 column
df_chada = df_chada.merge(
    ref_compound_df[['Metadata_Well', 'Metadata_Source', 'Metadata_Plate', 'Metadata_JCP2022']],
    on=['Metadata_Well', 'Metadata_Source', 'Metadata_Plate'],
    how='left'
)

# Display the updated df_chada
df_chada


,Metadata_Source,Metadata_Plate,Metadata_Well,Metadata_InChI,Metadata_PlateType,Metadata_Batch,Metadata_Is_Control,Embeddings_mean,Metadata_Row,Metadata_Col,Path_Embeddings,Metadata_JCP2022
0,source_11,EC000025,B03,InChI=1S/C25H31N3O3/c1-3-20-12-7-8-14-22(20)26...,COMPOUND,Batch1,False,"[0.78557825, 0.41657415, -2.1980793, -0.388481...",2,3,/projects/synsight/data/chada/well_embeddings/...,JCP2022_042808
1,source_2,1086289693,F20,InChI=1S/C16H15F3N2O3/c1-10-4-2-3-5-12(10)24-9...,COMPOUND,20211003_Batch_13,False,"[0.18799336, -0.4325749, -1.0657859, -0.159291...",6,20,/projects/synsight/data/chada/well_embeddings/...,JCP2022_029363
2,source_6,110000296324,I12,InChI=1S/C21H20FN3O2/c22-17-7-5-15(6-8-17)20-1...,COMPOUND,p210927CPU2OS48hw384exp029JUMP,False,"[-0.24563168, -0.4394592, -0.9976242, -0.27873...",9,12,/projects/synsight/data/chada/well_embeddings/...,JCP2022_025791
3,source_8,A1166159,A11,InChI=1S/C15H20ClN3O3/c1-15(14(21)18-6-8-22-2)...,COMPOUND,J4,False,"[0.4652346, -0.3344115, -2.3212726, -0.6073519...",1,11,/projects/synsight/data/chada/well_embeddings/...,JCP2022_028247
4,source_9,GR00004394,L12,"InChI=1S/C13H22N4O2S2/c14-21(18,19)12-9-15-13(...",COMPOUND,20211102-Run15,False,"[0.2912807, -0.3196573, -0.66259694, -0.220878...",12,12,/projects/synsight/data/chada/well_embeddings/...,JCP2022_071337
...,...,...,...,...,...,...,...,...,...,...,...,...
745874,source_3,J12440b,K21,"InChI=1S/C17H17N3O3S2/c21-17-8-7-13(25(22,23)2...",COMPOUND,CP_26_all_Phenix1,False,"[0.3275261, -0.44286105, -2.4580777, -0.320452...",11,21,/projects/synsight/data/chada/well_embeddings/...,JCP2022_029013
745875,source_8,A1170491,N21,InChI=1S/C2H6OS/c1-4(2)3/h1-2H3,COMPOUND,J2,True,"[0.21251413, -0.49264348, -1.3788915, -0.39903...",14,21,/projects/synsight/data/chada/well_embeddings/...,JCP2022_033924
745876,source_2,1086290361,N12,InChI=1S/C14H12N4O2S/c1-9-13(21-8-18-9)14(19)1...,COMPOUND,20210920_Batch_12,False,"[0.12376895, -0.33975706, -1.3941263, -0.56959...",14,12,/projects/synsight/data/chada/well_embeddings/...,JCP2022_080475
745877,source_3,SP32P35a,J22,InChI=1S/C19H27N5OS/c1-20-18(25)17-15(16-19(26...,COMPOUND,CP_29_all_Phenix1,False,"[0.09203631, -0.21997674, -2.5780022, -0.44948...",10,22,/projects/synsight/data/chada/well_embeddings/...,JCP2022_108539


In [79]:
df_chada = df_chada[common_columns + ['Path_Embeddings']]

In [80]:

def filter_and_save_embeddings_with_metadata_chada(
    df: pd.DataFrame,
    embedding_col: str,
    name: str,
    metadata_cols: List[str],
    storage_folder: Union[str, Path],
    num_jobs: int = -1,
) -> None:
    """
    Filters rows of a DataFrame to match a reference DataFrame, keeps relevant columns,
    loads embeddings into a single NumPy array, and saves metadata with matching indices.

    Args:
        df (pd.DataFrame): The DataFrame containing embedding paths and metadata.
        ref_compound_df (pd.DataFrame): Reference DataFrame to filter rows.
        embedding_col (str): The column containing paths to torch tensors.
        name (str): A name to identify the saved files.
        metadata_cols (List[str]): List of metadata columns to include.
        storage_folder (Union[str, Path]): Folder to save the outputs.
        num_jobs (int): Number of parallel jobs for embedding processing.
    """
    # Ensure the storage folder exists
    storage_folder = Path(storage_folder)
    storage_folder.mkdir(parents=True, exist_ok=True)

    # Validate embedding and metadata columns
    if embedding_col not in df.columns:
        raise ValueError(f"Column '{embedding_col}' not found in the DataFrame.")

    missing_metadata_cols = [col for col in metadata_cols if col not in df.columns]
    if missing_metadata_cols:
        raise ValueError(f"Missing metadata columns: {missing_metadata_cols}")

    # Filter rows based on the reference DataFrame
    filtered_df = df

    # Function to load an embedding
    def load_embedding(embedding_path):
        try:
            tensor = torch.load(embedding_path, weights_only=True)
            return tensor.numpy()
        except Exception as e:
            print(f"Error loading embedding at {embedding_path}: {e}")
            return None

    # Wrap embedding loading with tqdm for progress tracking
    embedding_paths = filtered_df[embedding_col].tolist()

    # Use tqdm to wrap Parallel processing
    embeddings = Parallel(n_jobs=num_jobs)(
        delayed(lambda path: (path, load_embedding(path)))(path)
        for path in tqdm(embedding_paths)
    )
    embeddings = [emb for _, emb in embeddings if emb is not None]

    # Filter out failed embeddings and update metadata accordingly
    valid_indices = [i for i, emb in enumerate(embeddings) if emb is not None]
    embeddings = np.array([embeddings[i] for i in valid_indices])
    filtered_metadata = filtered_df.iloc[valid_indices].drop(columns=[embedding_col])[metadata_cols]

    # Save embeddings as a single NumPy array
    embeddings_path = storage_folder / f"embeddings_{name}.npy"
    np.save(embeddings_path, embeddings)

    # Save filtered metadata as a Parquet file
    metadata_parquet_path = storage_folder / f"metadata_{name}.parquet"
    filtered_metadata.to_parquet(metadata_parquet_path, index=False)

    print(f"Embeddings saved to: {embeddings_path}")
    print(f"Metadata saved to: {metadata_parquet_path}")
    return filtered_metadata, embeddings




In [81]:
filtered_metadata, embeddings = filter_and_save_embeddings_with_metadata_chada(
    df=df_chada,
    embedding_col="Path_Embeddings",  
    name='chada',
    metadata_cols=common_columns,
    storage_folder=chada_path,
    num_jobs=79,
)

filtered_metadata.info()

100%|██████████| 745879/745879 [04:33<00:00, 2731.05it/s] 


Embeddings saved to: /projects/synsight/data/jump_embeddings/wells_embeddings/chada/embeddings_chada.npy
Metadata saved to: /projects/synsight/data/jump_embeddings/wells_embeddings/chada/metadata_chada.parquet
<class 'pandas.core.frame.DataFrame'>
Index: 745879 entries, 0 to 745878
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Metadata_Source   745879 non-null  object
 1   Metadata_Batch    745879 non-null  object
 2   Metadata_Plate    745879 non-null  object
 3   Metadata_Well     745879 non-null  object
 4   Metadata_JCP2022  745879 non-null  object
 5   Metadata_InChI    745879 non-null  object
dtypes: object(6)
memory usage: 39.8+ MB


### dino (s and g have the exact same lines)

#### dino s

In [50]:
df_dinov2_s_good = pd.read_parquet('/projects/synsight/data/dinov2_small/raw_well_embeddings/compounds_wells_em_dinov2s.parquet')

In [58]:
dinov2_s_path = base_path / 'dinov2_s'
df_dinov2_s_good.columns


Index(['Metadata_Source', 'Metadata_Batch', 'Metadata_Plate', 'Metadata_Well',
       'Metadata_JCP2022', 'Metadata_InChI', 'Metadata_Is_Control',
       'Metadata_Row', 'Metadata_Col', 'path_embedding'],
      dtype='object')

In [ ]:
filtered_metadata, embeddings = filter_and_save_embeddings_with_metadata(
    df=df_dinov2_s_good,
    ref_compound_df=ref_compound_df,
    embedding_col="path_embedding",  
    name='dinov2_s',
    metadata_cols=common_columns,
    storage_folder=dinov2_s_path,
    num_jobs=79,
)

filtered_metadata.info()

 14%|█▍        | 104517/745884 [14:22<1:36:52, 110.34it/s]

#### dino g

In [44]:
df_dinov2_g_good = pd.read_parquet('/projects/synsight/data/dinov2/raw_well_embeddings/compounds_wells_em_dinov2g.parquet')

In [59]:
dino_g_path = base_path / 'dinov2_g'
df_dinov2_g_good.columns


Index(['Metadata_Source', 'Metadata_Batch', 'Metadata_Plate', 'Metadata_Well',
       'Metadata_JCP2022', 'Metadata_InChI', 'Metadata_Is_Control',
       'Metadata_Row', 'Metadata_Col', 'path_embedding'],
      dtype='object')

In [46]:
filtered_metadata, embeddings = filter_and_save_embeddings_with_metadata(
    df=df_dinov2_g_good,
    ref_compound_df=ref_compound_df,
    embedding_col="path_embedding",  
    name='dinov2_g',
    metadata_cols=common_columns,
    storage_folder=dino_g_path,
    num_jobs=79,
)

filtered_metadata.info()

100%|██████████| 10000/10000 [01:00<00:00, 164.95it/s]


Embeddings saved to: /projects/synsight/data/jump_embeddings/wells_embeddings/chada/embeddings_dino_g.npy
Metadata saved to: /projects/synsight/data/jump_embeddings/wells_embeddings/chada/metadata_dino_g.parquet
<class 'pandas.core.frame.DataFrame'>
Index: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Metadata_Source   10000 non-null  object
 1   Metadata_Batch    10000 non-null  object
 2   Metadata_Plate    10000 non-null  object
 3   Metadata_Well     10000 non-null  object
 4   Metadata_JCP2022  10000 non-null  object
 5   Metadata_InChI    10000 non-null  object
dtypes: object(6)
memory usage: 546.9+ KB


### OpenPhenom

In [60]:
openphenom_path = base_path / 'openphenom'
df_phenom.columns

Index(['Metadata_Well', 'Metadata_Source', 'Metadata_Batch', 'Metadata_Plate',
       'Metadata_JCP2022', 'Metadata_InChI', 'Metadata_Is_Control',
       'Metadata_Row', 'Metadata_Col', 'path_embedding'],
      dtype='object')

In [61]:
filtered_metadata, embeddings = filter_and_save_embeddings_with_metadata(
    df=df_phenom,
    ref_compound_df=ref_compound_df,
    embedding_col="path_embedding",  
    name='openphenom',
    metadata_cols=common_columns,
    storage_folder=openphenom_path,
    num_jobs=79,
)

filtered_metadata.info()

100%|██████████| 745884/745884 [37:59<00:00, 327.19it/s]  


Embeddings saved to: /projects/synsight/data/jump_embeddings/wells_embeddings/openphenom/embeddings_openphenom.npy
Metadata saved to: /projects/synsight/data/jump_embeddings/wells_embeddings/openphenom/metadata_openphenom.parquet
<class 'pandas.core.frame.DataFrame'>
Index: 745884 entries, 0 to 745883
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Metadata_Source   745884 non-null  object
 1   Metadata_Batch    745884 non-null  object
 2   Metadata_Plate    745884 non-null  object
 3   Metadata_Well     745884 non-null  object
 4   Metadata_JCP2022  745884 non-null  object
 5   Metadata_InChI    745884 non-null  object
dtypes: object(6)
memory usage: 39.8+ MB
